# Demonstration of linear mosaic

Reads a CASA MeasurementSet containing 7 pointing observation of CASA at 8.4 GHz, forms linear mosaic of dirty images

In [ ]:
%matplotlib inline

import os
import sys

sys.path.append(os.path.join('..', '..'))

from data_models.parameters import arl_path

results_dir = arl_path('test_results')

from matplotlib import pylab

import numpy

from matplotlib import pyplot as plt

from data_models.polarisation import PolarisationFrame

from data_models.parameters import arl_path

from processing_library import copy_image
from processing_components import  create_visibility_from_ms, show_image, create_pb, create_image_from_visibility, \
    invert_2d

import logging

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))
mpl_logger = logging.getLogger("matplotlib") 
mpl_logger.setLevel(logging.WARNING) 


In [ ]:
pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'Greys'

In [ ]:
vis_list=create_visibility_from_ms(arl_path('data/vis/xcasa.ms'))

In [ ]:
# plot the amplitude of the visibility for each field
for field, vt in enumerate(vis_list):
    uvdist=numpy.sqrt(vt.data['uvw'][:,0]**2+vt.data['uvw'][:,1]**2)
    plt.clf()
    plt.plot(uvdist, numpy.abs(vt.data['vis']), '.')
    plt.xlabel('uvdist')
    plt.ylabel('Amp Visibility')
    plt.title('Field %d' % (field))
    plt.show()

Make the dirty images

In [ ]:
cellsize=0.00001

mid_frequency = [0.5 * (vis_list[0].frequency[0]+vis_list[1].frequency[0])]
total_bandwidth = [vis_list[0].channel_bandwidth[0]+vis_list[1].channel_bandwidth[0]]
model = create_image_from_visibility(vis_list[0], cellsize=cellsize, npixel=512, nchan=1,
                                     frequency=mid_frequency, channel_bandwidth=total_bandwidth,
                                     imagecentre=vis_list[0].phasecentre,
                                     polarisation_frame=PolarisationFrame('stokesIQUV'))
mosaic = copy_image(model)
mosaicsens = copy_image(model)
work = copy_image(model)

for vt in vis_list:
    beam = create_pb(model, telescope='VLA', pointingcentre=vt.phasecentre)
    beam.data /= numpy.max(beam.data)
    dirty, sumwt = invert_2d(vt, model)
    mosaic.data += dirty.data * beam.data
    mosaicsens.data += beam.data**2
    
show_image(mosaic, cm='Greys', title='Linear mosaic')
plt.show()
show_image(mosaicsens, cm='Greys', title='Linear mosaic sensitivity')
plt.show()
